# 02_comparacion_resultados — Baselines A/D

**Objetivo:** consolidar y analizar resultados de los tres baselines (reglas, TF‑IDF, transformer).  
**Exportables:** `data/02_baselines_comparacion.csv` con métricas clave (macro F1/Prec/Rec, n).


In [1]:
# === Paths / Globals (auto-detect) ===
from pathlib import Path
import pandas as pd
import re, unicodedata, os

# Rutas y entorno
BASE_PATH = Path.cwd()
if BASE_PATH.name == "notebooks":
    BASE_PATH = BASE_PATH.parent

DATA_PATH = BASE_PATH / "data"
FORK_PATH = BASE_PATH / "Spanish_Psych_Phenotyping_PY"

# Reuse existing globals if present (from your 02_baselines.ipynb)
DATA_PATH = Path(DATA_PATH) if 'DATA_PATH' in globals() else Path('data')
FORK_PATH = Path(FORK_PATH) if 'FORK_PATH' in globals() else Path('Spanish_Psych_Phenotyping_PY')
DATA_PATH.mkdir(exist_ok=True)


## 1) Cargar métricas y unificar

In [2]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path(DATA_PATH) if 'DATA_PATH' in globals() else Path('data')

paths = {
    "rule_based": {
        "eval": DATA_PATH/'rule_based_eval.csv',
        "report": DATA_PATH/'rule_based_classification_report.csv',
        "pred": DATA_PATH/'rule_based_predictions.csv',
    },
    "tfidf": {
        "eval": DATA_PATH/'tfidf_eval.csv',
        "report": DATA_PATH/'tfidf_classification_report.csv',
        "pred": DATA_PATH/'tfidf_predictions.csv',
    },
    "beto": {
        "eval": DATA_PATH/'beto_eval.csv',
        "report": DATA_PATH/'beto_classification_report.csv',
        "pred": DATA_PATH/'beto_predictions.csv',
    }
}

def pick(series, *names, default=None):
    """Devuelve el primer nombre presente en la serie."""
    for n in names:
        if n in series:
            return series[n]
    return default

rows = []
for name, ps in paths.items():
    if ps["eval"].exists():
        ev = pd.read_csv(ps["eval"]).iloc[0].to_dict()
        # Aceptar con o sin prefijo 'eval_'
        macro_f1       = pick(ev, "macro_f1", "eval_macro_f1")
        macro_prec     = pick(ev, "macro_precision", "eval_macro_precision")
        macro_rec      = pick(ev, "macro_recall", "eval_macro_recall")
        n_val          = pick(ev, "n")  # puede no existir en beto_eval
        if pd.isna(n_val) or n_val is None:
            # fallback: contar filas de predicciones si existe
            if ps.get("pred") and Path(ps["pred"]).exists():
                n_val = len(pd.read_csv(ps["pred"]))
        rows.append({"baseline": name,
                     "macro_f1": macro_f1,
                     "macro_precision": macro_prec,
                     "macro_recall": macro_rec,
                     "n": n_val})
    else:
        rows.append({"baseline": name, "macro_f1": None, "macro_precision": None, "macro_recall": None, "n": None})

comp = pd.DataFrame(rows, columns=["baseline","macro_f1","macro_precision","macro_recall","n"])
out_csv = DATA_PATH/'02_baselines_comparacion.csv'
comp.to_csv(out_csv, index=False, encoding='utf-8')
comp


,baseline,macro_f1,macro_precision,macro_recall,n
0,rule_based,0.529224,0.561122,0.537079,630.0
1,tfidf,0.913216,0.916667,0.909961,630.0
2,beto,0.855857,0.869106,0.845339,630.0


## 2) Comentarios y guía de interpretación

- **Rule-based**: traza de patrones: si macro-F1 es muy bajo y precisión alta pero recall bajo, suele indicar **cobertura insuficiente** de reglas ante typos/variantes locales. Útil para depurar fenómenos lingüísticos (ej.: negaciones, abreviaturas).
- **TF‑IDF (char)**: debe ser más **robusto a ruido**; si supera a reglas, confirma que los errores ortográficos afectan a las reglas.
- **Transformer**: si el dataset es suficiente y el preprocesamiento conservador, debería ser **competitivo** o superior; si no mejora, revisar **tamaño de datos**, **desbalance** o **ruido de etiquetas**.
- **Próximos pasos**: calibrar *thresholds*, *class weights*, *data cleaning* incremental; y documentar ejemplos de **falsos positivos/negativos** desde los CSV de predicciones para la discusión en tu presentación.
